In [1]:
# Module Importations


In [ ]:
# Project Module Importations

In [1]:
# Pyspark Import and Setup
import findspark
findspark.find()
findspark.init()
import pyspark

In [ ]:
# Constants (i.e. for saving figures)